In [1]:
import simtk.openmm.app as app
import simtk.openmm as mm
import simtk.unit as unit
from sys import stdout
from pdbfixer import PDBFixer

**Cargar el archivo PDB del disco y agregar residuos y átomos restantes**:

In [2]:
pdb = PDBFixer("BAK_model1.pdb")

**Agregar residuos, átomos e hidrógenos restantes**:

In [3]:
pdb.findMissingResidues()

pdb.findMissingAtoms()
pdb.addMissingAtoms()

pdb.addMissingHydrogens(7.0)

app.PDBFile.writeFile(pdb.topology, pdb.positions, open("BAK_model1_all.pdb", "w"))

**Especificar el campo de fuerza a utilizar para la simulación**:

In [4]:
forcefield =app.ForceField("amber14-all.xml", "amber14/tip3pfb.xml")

**Crear una descripción matematica completa del sistema por simular**:

In [5]:
system = forcefield.createSystem(pdb.topology, nonbondedMethod=app.NoCutoff, constraints=app.HBonds)

**Crear el integrador**:

Esta línea crea el integrador empleado para avanzar las ecuaciones de movimiento. 

LangevinIntegrator: realiza la dinámica de Langevin, y lo asigna a una variable llamada integrador.

Se declaran los valores de tres parámetros que son específicos de la dinámica de Langevin: la temperatura de simulación (300 K), el coeficiente de fricción (1 ps -1 ) y el tamaño del paso (2 fs).

In [6]:
integrator = mm.LangevinIntegrator(300*unit.kelvin, 1.0/unit.picoseconds, 2.0*unit.femtoseconds)
integrator.setConstraintTolerance(0.00001)

**Preparar la simulación**:

- Definir la plataforma (CUDA o CPU):

In [7]:
platform = mm.Platform.getPlatformByName('CUDA')
properties = {'CudaPrecision': 'mixed'}

- Crear el objeto simulación:

Esta línea combina la topología molecular, el sistema y el integrador para comenzar una nueva simulación.

In [8]:
simulation = app.Simulation(pdb.topology, system, integrator, platform, properties)

In [9]:
simulation.context.setPositions(pdb.positions)

(Esta línea especifica las posiciones iniciales de los átomos para la simulación, en este caso: las posiciones que se cargaron desde el archivo PDB.)

**Minimización de energía**:

In [10]:
print('Minimizing...')
simulation.minimizeEnergy(tolerance=0.1*unit.kilojoule/unit.mole, maxIterations=500)

Minimizing...
